In [1]:
 import sys
 print(sys.executable)
 print(sys.version)
 print(sys.version_info)

/home/antslab/anaconda3/envs/Leo_Env/bin/python
3.7.7 (default, Mar 26 2020, 15:48:22) 
[GCC 7.3.0]
sys.version_info(major=3, minor=7, micro=7, releaselevel='final', serial=0)


In [2]:
import findspark
findspark.init()
import pyspark
# from pyspark import SparkContext
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import databricks.koalas as ks
import pandas as pd
import numpy as np
# import koalas as ks

logFile = "../spark_data/data/Leo_Spark_Home/README.md"  # Should be some file on your system
appName = "Test App"
master = 'local'
# sc = SparkContext(master, appName)


In [3]:
conf = SparkConf().setAppName(appName).setMaster(master).set("spark.network.timeout","800s").set("spark.driver.memory","64g").set("spark.memory.storageFraction","0.3")
# sc = SparkContext(conf=conf)
# logData = sc.textFile(logFile).cache()
from pyspark.conf import SparkConf
spark = SparkSession.builder.master("local").appName("dionaea_analyzer").config(conf=conf).getOrCreate()
# spark = SparkSession.builder.master("local").appName("Test_spark").config(conf=SparkConf()).getOrCreate()
# spark = SparkSession.builder.appName(name="Test_spark").config().getOrCreate()

# spark.conf.set("spark.sql.execution.arrow.enabled", "true")
spark.version

In [5]:
%%time 
#6min
dir_path = '/home/antslab/spark_data/intermediate/honeypot/dionaea_2020-01-10/'
sdf = spark.read.json(dir_path+"*.json")
sdf.show()

+----+--------------------+---------+----------+--------------------+----+-------------------+
|  ip|          log_domain|log_level|message_ip|        message_noip|port|          timestamp|
+----+--------------------+---------+----------+--------------------+----+-------------------+
|null|python module.c:1028|  warning|      null|/usr/lib/dionaea/...|null|2020-01-10 00:09:47|
|null|python module.c:1029|  warning|      null|	 (time.time(), c...|null|2020-01-10 00:09:47|
|null|python module.c:1028|  warning|      null|/usr/lib/dionaea/...|null|2020-01-10 00:09:47|
|null|python module.c:1029|  warning|      null|	 attackid = self...|null|2020-01-10 00:09:47|
|null|python module.c:1028|  warning|      null|binding.pyx:1184 ...|null|2020-01-10 00:09:47|
|null|python module.c:1029|  warning|      null|              	 None|null|2020-01-10 00:09:47|
|null|python module.c:1003|  warning|      null|OperationalError ...|null|2020-01-10 00:09:47|
|null|python module.c:1028|  warning|      null|/u

In [6]:
%%time
kdf = sdf.to_koalas()
kdf

CPU times: user 9.81 ms, sys: 3.91 ms, total: 13.7 ms
Wall time: 117 ms


,ip,log_domain,log_level,message_ip,message_noip,port,timestamp
0,None,python module.c:1028,warning,None,/usr/lib/dionaea/python/dionaea/logsql.py:568 ...,NaN,2020-01-10 00:09:47
1,None,python module.c:1029,warning,None,"\t (time.time(), connection_type, con.transpor...",NaN,2020-01-10 00:09:47
2,None,python module.c:1028,warning,None,/usr/lib/dionaea/python/dionaea/logsql.py:621 ...,NaN,2020-01-10 00:09:47
3,None,python module.c:1029,warning,None,"\t attackid = self.connection_insert( icd, 'ac...",NaN,2020-01-10 00:09:47
4,None,python module.c:1028,warning,None,binding.pyx:1184 in dionaea.core.c_python_ihan...,NaN,2020-01-10 00:09:47
5,None,python module.c:1029,warning,None,\t None,NaN,2020-01-10 00:09:47
6,None,python module.c:1003,warning,None,OperationalError at disk I/O error,NaN,2020-01-10 00:09:47
7,None,python module.c:1028,warning,None,/usr/lib/dionaea/python/dionaea/logsql.py:568 ...,NaN,2020-01-10 00:09:47
8,None,python module.c:1029,warning,None,"\t (time.time(), connection_type, con.transpor...",NaN,2020-01-10 00:09:47
9,None,python module.c:1028,warning,None,/usr/lib/dionaea/python/dionaea/logsql.py:621 ...,NaN,2020-01-10 00:09:47


In [4]:
test_MDB_dir = '../spark_data/intermediate/honeypot/MDB_CNS_2019-12-11/'
sdf2 = spark.read.json(test_MDB_dir+"amun.events.json")
sdf2.show()

+--------------------+-----------+--------------------+----------+--------------------+--------------------+
|                 _id|    channel|               ident|normalized|             payload|           timestamp|
+--------------------+-----------+--------------------+----------+--------------------+--------------------+
|[5df0318090721a14...|amun.events|52da7d68-cba0-11e...|      true|[119.28.222.106, ...|[2019-12-11T00:00...|
|[5df0318090721a14...|amun.events|52da7d68-cba0-11e...|      true|[193.188.23.18, 6...|[2019-12-11T00:00...|
|[5df0318190721a14...|amun.events|52da7d68-cba0-11e...|      true|[181.65.210.130, ...|[2019-12-11T00:00...|
|[5df0318290721a14...|amun.events|52da7d68-cba0-11e...|      true|[47.74.223.87, 56...|[2019-12-11T00:00...|
|[5df0318290721a14...|amun.events|52da7d68-cba0-11e...|      true|[88.206.78.253, 4...|[2019-12-11T00:00...|
|[5df0318390721a14...|amun.events|52da7d68-cba0-11e...|      true|[34.70.182.59, 56...|[2019-12-11T00:00...|
|[5df0318390721a14.

In [6]:
kf2 = sdf2.to_koalas()
df2 = sdf2.toPandas()
kf2.head(20)

,_id,channel,ident,normalized,payload,timestamp
0,"(5df0318090721a14d015ba86,)",amun.events,52da7d68-cba0-11e8-ac82-3ca82a1c2b72,True,"(119.28.222.106, 14972, initial, 203.187.87.51...","(2019-12-11T00:00:00.083Z,)"
1,"(5df0318090721a14d015bb03,)",amun.events,52da7d68-cba0-11e8-ac82-3ca82a1c2b72,True,"(193.188.23.18, 64012, initial, 203.187.87.40,...","(2019-12-11T00:00:00.587Z,)"
2,"(5df0318190721a14d015bb20,)",amun.events,52da7d68-cba0-11e8-ac82-3ca82a1c2b72,True,"(181.65.210.130, 55732, initial, 203.187.87.26...","(2019-12-11T00:00:01.089Z,)"
3,"(5df0318290721a14d015bb28,)",amun.events,52da7d68-cba0-11e8-ac82-3ca82a1c2b72,True,"(47.74.223.87, 56980, initial, 203.187.87.38, 23)","(2019-12-11T00:00:02.280Z,)"
4,"(5df0318290721a14d015bb37,)",amun.events,52da7d68-cba0-11e8-ac82-3ca82a1c2b72,True,"(88.206.78.253, 49447, initial, 203.187.87.53,...","(2019-12-11T00:00:02.786Z,)"
5,"(5df0318390721a14d015bb4b,)",amun.events,52da7d68-cba0-11e8-ac82-3ca82a1c2b72,True,"(34.70.182.59, 56579, initial, 203.187.87.14, ...","(2019-12-11T00:00:03.287Z,)"
6,"(5df0318390721a14d015bb9b,)",amun.events,52da7d68-cba0-11e8-ac82-3ca82a1c2b72,True,"(119.28.222.106, 15606, initial, 203.187.87.51...","(2019-12-11T00:00:03.796Z,)"
7,"(5df0318490721a14d015bbb6,)",amun.events,52da7d68-cba0-11e8-ac82-3ca82a1c2b72,True,"(34.66.68.212, 60833, initial, 203.187.87.32, ...","(2019-12-11T00:00:04.298Z,)"
8,"(5df0318490721a14d015bbd2,)",amun.events,52da7d68-cba0-11e8-ac82-3ca82a1c2b72,True,"(186.53.131.130, 62111, initial, 203.187.87.58...","(2019-12-11T00:00:04.801Z,)"
9,"(5df0318590721a14d015bbde,)",amun.events,52da7d68-cba0-11e8-ac82-3ca82a1c2b72,True,"(34.66.68.212, 60835, initial, 203.187.87.36, ...","(2019-12-11T00:00:05.313Z,)"


In [17]:
sdf2.collect()[0][4]

Row(attackerIP='119.28.222.106', attackerPort=14972, connectionType='initial', victimIP='203.187.87.51', victimPort=80)

In [24]:
df2.iloc[0,5][0]

'2019-12-11T00:00:00.083Z'

In [22]:
kf2.iloc[0,4][0]

'119.28.222.106'

In [31]:
kf2.columns[4]

'payload'

In [56]:
df2['test'] = df2['payload'].map(test_func)

In [57]:
df2[df2['test'] == 22]

,_id,channel,ident,normalized,payload,timestamp,payload2,test


In [63]:
df2['payload2'] = df2['payload'].astype('str')
df2['connectionType' in df2.payload2]

KeyError: False

In [44]:
df2.payload2[0]

"Row(attackerIP='119.28.222.106', attackerPort=14972, connectionType='initial', victimIP='203.187.87.51', victimPort=80)"

In [26]:
sdf2['payload']

Column<b'payload[0]'>

In [55]:
def test_func(s):
    if s[2] != 'initial':
        print(s)
        return(22)

In [59]:
import pickle
ips = pickle.load(open('..//spark_data/intermediate/pcap/cht_0110_top10.pk','rb'))
ips

[Row(ip.src=['210.242.154.92'], count(1)=10421545),
 Row(ip.src=['210.242.196.178'], count(1)=8256328),
 Row(ip.src=['5.188.211.35'], count(1)=6773568),
 Row(ip.src=['210.242.49.2'], count(1)=6134645),
 Row(ip.src=['185.176.27.250'], count(1)=5548999),
 Row(ip.src=['210.109.97.141'], count(1)=5016321),
 Row(ip.src=['210.212.235.51'], count(1)=3655578),
 Row(ip.src=['92.246.76.77'], count(1)=3140992),
 Row(ip.src=['210.14.69.44'], count(1)=2413578),
 Row(ip.src=['81.22.45.190'], count(1)=2375558)]

In [61]:
ips[0]

Row(ip.src=['210.242.154.92'], count(1)=10421545)